In [1]:
from __future__ import division

import sys
sys.path.append("../Modules/")
sys.path.append("../")
import os
import pandas as pd

import numpy as np
from scipy import signal as sig

import performFun as pF
import catchE1Funs as expFun
import recalibration as rc

from plottingFuns import *
from analysisParameters import loadParameters

bkP.output_notebook()

fileTime = '2016-4-27-13-28'

analysisParameters = loadParameters(fileTime)

Loading BokehJS ...

**** Loading 2 **** 


In [2]:
startFresh = True
loadProcessed = False

##############################################################################
##############################################################################
### loadSessionDict
# We can just tell members of the audience that any way they can get the data into a dataframe is OK
sessionDict = pF.loadSessionDict(analysisParameters,startFresh=startFresh,loadProcessed = loadProcessed)

Loading experiment config file: ../Data/2016-4-27-13-28/gd_pilot.cfg
Experiment config file parsed correctly
Loading system config file: ../Data/2016-4-27-13-28/PERFORMVR.cfg
System config file parsed correctly


# Calc cyc gaze direction

### Create cyclopean gaze vector

I'm pretending we dont' have cycEyeInHead.  Why?  I don't think the eye tracker gives it to you for free.  Lets begin by dropping it.

In [3]:
sessionDict['raw'] = sessionDict['raw'].drop('cycEyeInHead',axis=1,level=0)

### Take the average of the two vectors, then normalize

In [4]:
# Wow.  This method is actually quite slow compared to the numpy method.  I'm surprised!
cycEyeInHeadDf = sessionDict['raw'].apply(lambda x: (x['rightEyeInHead'] + x['leftEyeInHead'])/2.0 ,axis=1)
cycEyeInHeadDf[1:5]

,X,Y,Z
frameNum,,,
1,0.204464,0.102350,0.973102
2,0.206678,0.101886,0.972704
3,0.207992,0.101454,0.972478
4,0.209539,0.101647,0.972151


In [5]:
cycEyeInHeadDf = np.add(sessionDict['raw']['rightEyeInHead'],sessionDict['raw']['leftEyeInHead'])/2
cycEyeInHeadDf.columns = pd.MultiIndex.from_tuples([('cycEyeInHead','X'),('cycEyeInHead','Y'),('cycEyeInHead','Z')])
cycEyeInHeadDf[1:5]

cycEyeInHead                    
                    X         Y         Z
frameNum                                 
1            0.204464   0.10235  0.973102
2            0.206678  0.101886  0.972704
3            0.207992  0.101454  0.972478
4            0.209539  0.101647  0.972151

### Normalize the vector

In [6]:
def normalizeDirection(xyz):
    return xyz/np.linalg.norm(xyz)

cycEyeInHeadDf = cycEyeInHeadDf.apply(lambda xyz: normalizeDirection(xyz), axis=1)

### Add new dataframe to sessionDict['processed']

In [7]:
sessionDict['processed'] = pd.concat([sessionDict['processed'],cycEyeInHeadDf],axis=1,verify_integrity=1) 

# Now that we have the cyclopean gaze direction, we can apply our head transformation matrix to position it in the real world

## Note that this requires that we know what direction is 'forward' in the SMI glasses space, and in our own head's local coordinate system.  I'm fudging it right now because I haven't looked this up in a while

Note that this operation requires information spread across two dataframes.
So, as a first step, we must create a temporary dataframe with the columns of data that we will need of our operation

In [8]:
newDf = pd.concat([sessionDict['processed']['cycEyeInHead'],sessionDict['raw']['viewMat']],
          axis=1,keys=['cycEyeInHead','viewMatrix'])

newDf[1:5]

cycEyeInHead                     viewMatrix                          \
             X         Y         Z          0          1         2  3   
1     0.204545  0.102391  0.973487   0.979573   0.040979  0.196869  0   
2     0.206756  0.101924  0.973069   0.979625  0.0410081  0.196602  0   
3     0.208068  0.101491  0.972835   0.979673   0.040977  0.196372  0   
4     0.209611  0.101681  0.972483   0.979703  0.0408316  0.196252  0   

                                                                        \
           4         5          6  7         8          9        10 11   
1 -0.0596425   0.99417  0.0898266  0 -0.192041 -0.0997335  0.976306  0   
2 -0.0596815  0.994151  0.0900157  0 -0.191761 -0.0999152  0.976342  0   
3 -0.0596708   0.99413  0.0902438  0 -0.191522  -0.100127  0.976368  0   
4 -0.0595599  0.994114  0.0904943  0 -0.191402  -0.100346  0.976369  0   

                                   
         12       13        14 15  
1 -0.914088  1.70349  0.720328  1  
2  -0.91407  1.70366  0.719961  1  
3 -0.914259  1.70371  0.719824  1  
4 -0.914497  1.70367  0.719849  1

### Now, lets apply a function to each row that multiplies the data by the transform.  This should be a function.

In [9]:
# THis is the. only working implementation I've found. Ungh.
for fr in range(0,10): #range(len(sessionDict['raw']['viewMat'])):
    mat = sessionDict['raw']['viewMat'].values[fr].reshape([4,4]).T
    xyzw = np.append(sessionDict['processed']['cycEyeInHead'].values[fr],True) # The True converst into a 1:  [x y z 1]
    print np.dot(mat,xyzw)[0:3]


[-0.9072305704493391 1.716757568939831 1.720213239565728]
[-0.9067774000271243 1.7165759648058603 1.7202152346190756]
[-0.9042063656047987 1.7162423558820168 1.7198331128588706]
[-0.9027951030171278 1.7157266652739223 1.7196858774406332]
[-0.9013320802732305 1.7157315702365588 1.7196894562060985]
[-0.9013708781146015 1.7165969015120088 1.7197110387079477]
[-0.9018280666591405 1.7158919780642523 1.7200332504712368]
[-0.9019543229859046 1.7157196114810356 1.719974760701407]
[-0.9023072049594887 1.7155118580662545 1.7197011376281313]
[-0.9022277551428676 1.7160278677684453 1.7196385614633136]


In [10]:
out_fr_XYZ = [np.dot(sessionDict['raw']['viewMat'].values[fr].reshape([4,4]).T,
                     np.append(sessionDict['processed']['cycEyeInHead'].values[fr],1)) for fr in range(0,10)]

out_fr_XYZ

[array([-0.9072305704493391, 1.716757568939831, 1.720213239565728, 1.0], dtype=object),
 array([-0.9067774000271243, 1.7165759648058603, 1.7202152346190756, 1.0], dtype=object),
 array([-0.9042063656047987, 1.7162423558820168, 1.7198331128588706, 1.0], dtype=object),
 array([-0.9027951030171278, 1.7157266652739223, 1.7196858774406332, 1.0], dtype=object),
 array([-0.9013320802732305, 1.7157315702365588, 1.7196894562060985, 1.0], dtype=object),
 array([-0.9013708781146015, 1.7165969015120088, 1.7197110387079477, 1.0], dtype=object),
 array([-0.9018280666591405, 1.7158919780642523, 1.7200332504712368, 1.0], dtype=object),
 array([-0.9019543229859046, 1.7157196114810356, 1.719974760701407, 1.0], dtype=object),
 array([-0.9023072049594887, 1.7155118580662545, 1.7197011376281313, 1.0], dtype=object),
 array([-0.9022277551428676, 1.7160278677684453, 1.7196385614633136, 1.0], dtype=object)]

## I really want to solve this in a way that will allow use of the apply command.

### Here are some failed attempts

In [11]:
# Attempt one
#newDf.apply(lambda row: np.dot(row['viewMatrix'].values.reshape([4,4]).T,[0,0,1.0,0]),axis=1)



# Attempt two
#def multByMatrix(matrix_fr,xyz_fr,translate = True):       
#     out_fr_XYZ = np.array([ np.dot(matrix_fr.values[fr].reshape([4,4]).T,
#                                    np.append(xyz_fr.values[fr],translate)) 
#                            for fr in range(len(xyz_fr))])



### Eureeka!  It turns out that the output from the function must be a tuple, or you get the error....
ValueError: Shape of passed values is (40415, 3), indices imply (40415, 19)


In [12]:
# Fuck it!  We'll do it in a for loop.
def multByMatrix(matrix,xyz,translate = True):
        
    mat = matrix.values.reshape([4,4]).T
    xyzw = np.append(xyz,True)
    xyzOut = np.dot(mat,xyzw)
    
    # It seems to be an issue of the return type
    #return xyzOut[0:3] # THis does not work
    #return [1,2,3] # THis does

    #Eureeka!  You must return a tuple. This is pretty fast.
    return (xyzOut[0],xyzOut[1],xyzOut[2])



tranformedXYZ = newDf.apply(lambda row: multByMatrix(row['viewMatrix'],row['cycEyeInHead']),axis=1)
tranformedXYZ[1:10]

1    (-0.906777400027, 1.71657596481, 1.72021523462)
2    (-0.904206365605, 1.71624235588, 1.71983311286)
3    (-0.902795103017, 1.71572666527, 1.71968587744)
4    (-0.901332080273, 1.71573157024, 1.71968945621)
5    (-0.901370878115, 1.71659690151, 1.71971103871)
6    (-0.901828066659, 1.71589197806, 1.72003325047)
7     (-0.901954322986, 1.71571961148, 1.7199747607)
8    (-0.902307204959, 1.71551185807, 1.71970113763)
9    (-0.902227755143, 1.71602786777, 1.71963856146)
dtype: object

# Do this for a few more things.  Plot. Success!